In [2]:
import pandas as pd

In [41]:
features = pd.read_csv(
    r"../../data/clean_market_data/features.csv", index_col=0)
features.head()

,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,2015-01-14,2015-01-15,...,2021-10-18,2021-10-19,2021-10-20,2021-10-21,2021-10-22,2021-10-25,2021-10-26,2021-10-27,2021-10-28,2021-10-29
Tick,,,,,,,,,,,,,,,,,,,,,
CSCO,-0.897344,-0.951685,-0.331614,0.589535,-0.108935,0.433686,0.178569,-0.389928,0.287356,-2.420791,...,0.272525,0.813892,0.716849,-0.624558,-0.792077,0.054434,0.449964,0.089078,0.017788,-0.621446
ISRG,-1.000226,-1.608707,-0.689722,-0.217649,0.832386,-1.902180,0.512597,-1.943607,2.695134,-2.007032,...,0.596374,-0.535328,0.532541,-0.756822,0.520968,0.999440,0.573973,0.083171,1.823959,2.057369
LBTYK,-0.820684,-4.690083,-0.021682,-0.777539,-0.602286,-0.325030,0.108388,-0.021511,0.151843,-1.080847,...,0.376973,1.360543,0.739997,-0.536732,0.302827,-0.301710,-1.276456,-2.312140,-0.381549,0.069398
VRTX,2.583978,-1.014147,-3.256118,1.548564,1.614755,-1.244445,-3.820831,0.166110,1.655125,-2.483855,...,-0.931505,1.840056,-0.268832,0.835401,-1.740108,0.801003,-0.513456,-1.871033,0.447043,1.648986
GILD,-0.241745,1.627471,-0.832741,0.171187,1.488095,-1.265454,-1.584984,-4.555719,3.395836,-1.913395,...,-2.370368,1.363639,-0.074732,-0.149051,0.178777,0.639501,-0.353094,-1.751034,0.959519,-1.547806


In [42]:
sectors = pd.read_csv(
    r"../../data/raw_sectors/raw_sectors.csv", index_col=0)
sectors.head()

,Symbol,SECTOR
1,CPRT,Commercial Services
2,TMUS,Communications
3,TSLA,Consumer Durables
4,ATVI,Consumer Durables
5,EA,Consumer Durables


In [43]:
'''
Combine features and their corresponding sectors
'''
merge_sectors = features.merge(sectors, how='inner', left_index=True, right_on='Symbol').reset_index(drop=True)
print(merge_sectors.shape)


(95, 1722)


In [65]:
'''
We only care sectors that are bigger than or equals to expected average size
'''
count_map = dict(merge_sectors['SECTOR'].value_counts())
merge_sectors['SECTOR WEIGHT'] = merge_sectors['SECTOR'].map(count_map)

WEIGHT_TRESHOLD = int(merge_sectors.shape[0] / len(count_map))

sector_cluster = merge_sectors[merge_sectors['SECTOR WEIGHT'] > WEIGHT_TRESHOLD]
sector_cluster.drop('SECTOR WEIGHT', axis=1, inplace=True)


/Users/lin-kuan-you/miniconda/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [66]:
meta_data = sector_cluster.iloc[:, -2:]
meta_data.columns = ["Tick", "Cluster"]

sector_cluster = pd.concat([meta_data, sector_cluster.iloc[:, :-2]], axis=1)


In [68]:
sector_cluster

,Tick,Cluster,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,...,2021-10-18,2021-10-19,2021-10-20,2021-10-21,2021-10-22,2021-10-25,2021-10-26,2021-10-27,2021-10-28,2021-10-29
0,CSCO,Technology Services,-0.897344,-0.951685,-0.331614,0.589535,-0.108935,0.433686,0.178569,-0.389928,...,0.272525,0.813892,0.716849,-0.624558,-0.792077,0.054434,0.449964,0.089078,0.017788,-0.621446
1,ISRG,Health Technology,-1.000226,-1.608707,-0.689722,-0.217649,0.832386,-1.902180,0.512597,-1.943607,...,0.596374,-0.535328,0.532541,-0.756822,0.520968,0.999440,0.573973,0.083171,1.823959,2.057369
2,LBTYK,Consumer Services,-0.820684,-4.690083,-0.021682,-0.777539,-0.602286,-0.325030,0.108388,-0.021511,...,0.376973,1.360543,0.739997,-0.536732,0.302827,-0.301710,-1.276456,-2.312140,-0.381549,0.069398
3,VRTX,Health Technology,2.583978,-1.014147,-3.256118,1.548564,1.614755,-1.244445,-3.820831,0.166110,...,-0.931505,1.840056,-0.268832,0.835401,-1.740108,0.801003,-0.513456,-1.871033,0.447043,1.648986
4,GILD,Health Technology,-0.241745,1.627471,-0.832741,0.171187,1.488095,-1.265454,-1.584984,-4.555719,...,-2.370368,1.363639,-0.074732,-0.149051,0.178777,0.639501,-0.353094,-1.751034,0.959519,-1.547806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,SPLK,Technology Services,-0.876750,-2.470828,-2.380120,-1.488614,2.778269,-1.503761,-0.591922,-0.744718,...,-0.024400,1.012630,0.917046,1.095809,0.005968,1.190044,-0.958577,-2.420803,1.112544,0.690332
90,CTSH,Technology Services,-0.415960,-3.020198,-2.168492,2.811956,1.968208,-0.788268,0.222104,-1.168095,...,0.573683,0.630597,-0.552757,0.139294,-0.428531,-0.240054,-0.681045,-0.829507,-3.374153,1.521058
91,MU,Electronic Technology,-1.390470,-2.679344,-3.607037,1.807795,3.409081,-0.681278,-2.253781,-3.383458,...,0.373079,0.296864,1.592262,1.015456,-1.459641,-1.235278,-0.619865,-0.408638,0.418532,1.038162
92,CDW,Technology Services,-1.134424,-0.662441,-1.449695,1.195683,1.425720,-0.139819,-0.111891,0.833794,...,4.056543,0.476598,0.175477,1.205653,0.482812,-0.020883,-1.981484,-2.170228,0.589665,1.302575


In [69]:
sector_cluster.to_csv(r'../../data/cluster_results/sector_cluster_result.csv')